## Weather

In [1]:
import requests
import pandas as pd
from datetime import datetime

In [5]:
df = pd.read_csv('data/ithaca_weather_data.csv')

In [6]:
def get_weather(timestamp, weather_df):
    previous_hour = datetime.fromtimestamp(timestamp).replace(microsecond=0, second=0, minute=0).timestamp()
    return weather_df.loc[df['dt'] == previous_hour]

In [7]:
get_weather(1582882844, df)

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
389589,1582880400,2020-02-28 09:00:00 +0000 UTC,-18000,Ithaca,42.443961,-76.501881,20.19,5.94,17.01,23.0,...,250,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n


## Elevation 

In [11]:
with open('./keys/google_api_key.txt') as key_file:
    ELEVATION_KEY = key_file.read()
    
stops_df = pd.read_csv('./data/tcat-ny-us/stops.txt')

In [6]:
def get_elevation(lat, long):
    elevation_url = 'https://maps.googleapis.com/maps/api/elevation/json?locations={},{}&key={}'\
        .format(lat, long, ELEVATION_KEY)
    r = requests.get(elevation_url)
    if r.ok:
        return r.json()['results'][0]['elevation']
    raise RuntimeError('Error connecting to API')

In [7]:
stops_df['stop_elevations'] = [get_elevation(row[1]['stop_lat'], row[1]['stop_lon']) \
                               for row in stops_df.iterrows()]

In [8]:
stops_df.to_csv('stops-with-elevations.txt')